## Этап 1. Установка библиотек и импорт данных

In [ ]:
# скачиваем необходимые базовые библиотеки для работы
!pip install pandas numpy scikit-learn

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import glob
import os

Подключаемся в Google Drive, чтобы работать с файлами из Google Colaboratory


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Указываем путь к корневым папкам с данными test и train

__Обратите внимание:__ если вы создали папку с названием отличным от _"Хакатон Альфа банка"_, то необходимо поменять путь к директории и ссылки ниже

In [3]:
# путь к папке с тренировочными данными
path_train = '/content/drive/MyDrive/Хакатон Альфа-банка. Кейс 2/train'
# путь к папке с тестовыми данными
path_test = '/content/drive/MyDrive/Хакатон Альфа-банка. Кейс 2/test'

Получим список файлов в каждой из папок с данными

In [4]:
print(*[f'Список файлов в репозитории {x.split("/")[-1]}: {sorted(os.listdir(x))}' for x in [path_train, path_test]], sep='\n')

Список файлов в репозитории train: ['train_1.csv', 'train_10.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv', 'train_5.csv', 'train_6.csv', 'train_7.csv', 'train_8.csv', 'train_9.csv']
Список файлов в репозитории test: ['test_1.csv', 'test_10.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv', 'test_5.csv', 'test_6.csv', 'test_7.csv', 'test_8.csv', 'test_9.csv']


In [22]:
# получим список файлов в папке train
filenames_train = glob.glob(path_train + "/*.csv")

# создадим список для записи считанных файлов
data_files_train = []

# Инициализируем PCA для уменьшения размерности данных
pca = PCA(n_components=30, random_state=42)

data = pd.read_csv(filenames_train[0]).iloc[:, :103]

base_info = data[['target', 'smpl', 'id']]

pca.fit(data.drop(['target', 'smpl', 'id'], axis=1))

transformed_data = pd.DataFrame(pca.transform(data.drop(['target', 'smpl', 'id'], axis=1)))

result = pd.concat([base_info, transformed_data], ignore_index=True, axis=1)

new_columns = [f'feature_{i}' for i in range(1, 31)]

result.columns = ['target', 'smpl', 'id', *new_columns]

display(result.head())

# # выведем все файлы без объединения, чтобы избежать проблем с данными
# for filename in filenames_train[1:]:
#     data = pd.read_csv(filename).iloc[:, :78]
#     # data_files_train.append(data)
#     pca = PCA(n_components=20)
#     data = pca.fit_transform(data.drop(['target', 'smpl', 'id'], axis=1))
#     display(pd.DataFrame(data).head())
#     break

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30
0,0,train,0,5.136995,-2.849889,3.141443,-0.380265,0.923523,0.741111,0.375306,...,2.166438,-1.068913,-1.149858,0.353038,-0.364563,1.128543,1.168802,0.771338,-0.327063,-0.687300
1,0,train,1,-10.033835,-3.302581,0.654214,0.333623,0.175660,0.678814,-0.972199,...,-0.165227,0.313222,0.172487,1.803914,0.499972,0.623897,-1.269285,1.057380,0.773964,0.045677
2,0,train,2,5.171689,1.860613,1.229074,1.202280,2.765953,-0.391638,-0.598722,...,1.426717,-0.756269,-0.306375,0.925138,1.195234,-0.174030,-0.219223,0.143836,-1.002381,-0.894884
3,0,train,3,5.141020,3.685072,0.107632,-0.720552,1.025957,-0.238555,2.172228,...,-0.477358,-1.682262,1.070480,2.405650,0.486933,-1.876178,0.881214,-1.026250,-0.713084,-1.761063
4,0,train,4,5.232097,2.317622,0.405818,0.787412,-0.392777,-0.005067,1.170386,...,-0.076451,1.133021,-1.649373,0.722521,0.502443,-0.127449,0.031995,-0.479482,-0.174191,-0.523799


In [ ]:
# Объединим данные в один датасет
train_data = pd.concat(data_files_train)

X_train = train_data.drop(['target', 'smpl'], axis=1)
y_train = train_data['target']

# создадим объект класса PCA для уменьшения размерности данных
pca = PCA(n_components=20)
pca.fit(X_train)

# уменьшим размерность тренировочных данных
X_train_pca = pca.transform(X_train)

# Выведем первые 5 строк данных
display(X_train_pca.head())
# Проверим, что перед нами только тренировочные данные
display(X_train_pca['smpl'].value_counts())

In [ ]:
# Удалим ненужные переменные для очистки памяти
del data, data_files_train, filenames_train, path_train

In [ ]:
# получим список файлов в папке test
filenames_test = glob.glob(path_test + "/*.csv")

# создадим список для записи считанных файлов
data_files_test = []

# выведем все файлы без объединения, чтобы избежать проблем с данными
for filename in filenames_test:
    data = pd.read_csv(filename).iloc[:, :77]
    data_files_test.append(data)

In [ ]:
# Объединим данные в один датасет
test_data = pd.concat(data_files_test)
# Выведем первые 5 строк данных
display(test_data.head())
# Проверим, что перед нами только тестовые данные
display(test_data['smpl'].value_counts())

,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,test,4490468,0.177531,-1.152211,1.846728,0.008738,-0.431573,-0.263363,-0.224663,-0.333859,...,-0.725486,-0.620943,0.0,-0.496112,0.049236,0.494497,0.280554,-1.478610,0.073437,-1.419953
1,test,4490469,0.094399,-0.008647,0.505909,-1.539033,-1.108592,0.649972,-0.186942,1.166843,...,-0.795358,-0.552598,0.0,-2.874008,0.418197,1.530512,0.420642,0.535637,0.204647,0.826041
2,test,4490470,1.307947,0.370070,1.139814,-0.535084,0.914847,0.325444,0.806360,0.317192,...,2.110525,-0.621757,0.0,0.169976,1.387462,-0.525675,1.304250,-0.047983,0.174501,-0.036596
3,test,4490471,0.645874,-1.677962,0.910589,1.057324,-0.107759,-0.145189,-0.418729,-1.128467,...,-0.409737,-0.366802,0.0,-0.865381,-0.673515,0.559729,0.163176,-1.788871,0.982500,-1.550914
4,test,4490472,0.075710,0.709553,-1.667854,1.519592,0.016205,0.585399,0.993821,1.755325,...,-0.705051,0.995858,0.0,-0.452319,-1.292220,0.863420,0.286871,-2.725734,0.377897,-3.165806


,count
smpl,
test,509532


## Этап 2. Работа с моделью

In [ ]:
# Отделяем признаки и таргет во всех файлах
X_train = train_data.drop(['target', 'smpl'], axis=1)
y_train = train_data['target']

# обучаем модель на всех тренировочных данных
gbdt_clf = HistGradientBoostingClassifier(min_samples_leaf=1,
                                          max_depth=20,
                                          max_iter=125,
                                          random_state=42).fit(X_train, y_train)

## Этап 3. Предсказание для тестовых данных


Сделаем предсказание на тестовых данных

In [ ]:
# выделяем часть с данными для обучения на тестовых данных
X_test = test_data.drop('smpl', axis=1)
X_test_pca = pca.transform(X_test)

# получаем предсказания для данных
prediction = gbdt_clf.predict_proba(X_test)

# объединяем итоговое предсказание
X_test['target'] = pd.Series(prediction[:, 1])

# сохраняем предсказание
X_test[['id', 'target']].to_csv('baseline2_submission.csv', index=False)